In [6]:
from __future__ import print_function

import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.activations import relu
from tensorflow.keras.regularizers import l2
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import cifar100, cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

print("Packages Loaded")

Packages Loaded


In [7]:
# Now import Cifar-10 data and process it.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

print('Train: x=%s, y=%s' % (x_train.shape, y_train.shape))
print('Test: x=%s, y=%s' % (x_test.shape, y_test.shape))

Train: x=(50000, 32, 32, 3), y=(50000, 10)
Test: x=(10000, 32, 32, 3), y=(10000, 10)


In [8]:
# Now import Cifar-100 data and process it.
(x100_train, y100_train), (x100_test, y100_test ) = cifar100.load_data()
print('Unprocessed')
print('Train: x=%s, y=%s' % (x100_train.shape, y100_train.shape))
print('Test: x=%s, y=%s' % (x100_test.shape, y100_test.shape))
print('')

# Fill in the rest.
#x100_train = x100_train.reshape(50000, 3072)
#x100_test = x100_test.reshape(10000, 3072)
x100_train = x100_train.astype('float32')
x100_test = x100_test.astype('float32')
x100_train /= 255
x100_test /= 255

y100_train = tf.keras.utils.to_categorical(y100_train, 100)
y100_test = tf.keras.utils.to_categorical(y100_test, 100)

print('Processed')
print('Train: x=%s, y=%s' % (x100_train.shape, y100_train.shape))
print('Test: x=%s, y=%s' % (x100_test.shape, y100_test.shape))


Unprocessed
Train: x=(50000, 32, 32, 3), y=(50000, 1)
Test: x=(10000, 32, 32, 3), y=(10000, 1)

Processed
Train: x=(50000, 32, 32, 3), y=(50000, 100)
Test: x=(10000, 32, 32, 3), y=(10000, 100)


In [22]:
# Our baseline model for this lab.
model = Sequential()

model.add(Dense(64, activation='relu', input_shape=(3072,)))

model.add(Dense(64, activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(10, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

# Notice that you can reshape the data for an MLP inside the fit call without changing it globally!
model.fit(x_train.reshape(50000, 3072), y_train, epochs=20, batch_size=64, validation_data=(x_test.reshape(10000, 3072), y_test))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 64)                196672    
_________________________________________________________________
dense_16 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_17 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_18 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_19 (Dense)             (None, 10)                650       
Total params: 209,802
Trainable params: 209,802
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 92s 2m

In [30]:
# Do the same for CIFAR-100.
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(3072,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(100, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

model.fit(x100_train.reshape(50000, 3072), y100_train, epochs=20, batch_size=64, validation_data=(x100_test.reshape(10000, 3072), y100_test))


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 64)                196672    
_________________________________________________________________
dense_41 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_42 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_43 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_44 (Dense)             (None, 100)               6500      
Total params: 215,652
Trainable params: 215,652
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 21s 41

In [8]:
# Create the CIFAR-10 CNN baseline here.
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))       
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))   
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()
          
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# Fit data to model
history = model.fit(x_train, y_train,
            batch_size = 64,
            epochs = 10,      
            validation_data=(x_test, y_test))
          

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 12, 12, 32)        9248      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 10, 10, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 800)              

In [9]:
# Create the CIFAR-100 CNN baseline here.

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))       
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))   
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='softmax'))
model.summary()
          
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# Fit data to model
history = model.fit(x100_train, y100_train,
            batch_size = 64,
            epochs = 10,      
            validation_data=(x100_test, y100_test))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 12, 12, 32)        9248      
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 10, 10, 32)        9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 800)              

In [10]:
# Change the Convolutional layers to 64 filters for CIFAR-10.

model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))       
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))   
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()
          
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# Fit data to model
history = model.fit(x_train, y_train,
            batch_size = 64,
            epochs = 10,      
            validation_data=(x_test, y_test))
          

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 30, 30, 64)        1792      
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 12, 12, 64)        36928     
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 10, 10, 64)        36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 1600)             

In [11]:
# Change the Convolutional layers to 16 filters for CIFAR-10.

model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3)))
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))       
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))   
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()
          
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# Fit data to model
history = model.fit(x_train, y_train,
            batch_size = 64,
            epochs = 10,      
            validation_data=(x_test, y_test))
          

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 30, 30, 16)        448       
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 28, 28, 16)        2320      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 12, 12, 16)        2320      
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 10, 10, 16)        2320      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 400)              

In [5]:
# Remove the Max Pooling Layers for CIFAR-10.

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))       
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu')) 
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()
          
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# Fit data to model
history = model.fit(x_train, y_train,
            batch_size = 64,
            epochs = 10,      
            validation_data=(x_test, y_test))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               9437696   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0

In [5]:
# Remove the Max Pooling Layers for CIFAR-100.

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))       
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu')) 
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='softmax'))
model.summary()
          
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# Fit data to model
history = model.fit(x100_train, y100_train,
            batch_size = 64,
            epochs = 10,      
            validation_data=(x100_test, y100_test))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               9437696   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)              

In [3]:
# Data Augmentation.
batch_size = 64
num_classes = 10
epochs = 40
data_augmentation = True

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        zca_epsilon=1e-06,
        rotation_range=0,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0., 
        horizontal_flip=True,
        vertical_flip=False, 
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

    datagen.fit(x_train)

    model.fit(datagen.flow(x_train, y_train,
                        batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test))

scores = model.evaluate(x_test, y_test, verbose=0)

Using real-time data augmentation.
  ...
    to  
  ['...']
Train for 782 steps, validate on 10000 samples
Epoch 1/40
782/782 [==============================] - 97s 124ms/step - loss: 1.8367 - accuracy: 0.3264 - val_loss: 1.3982 - val_accuracy: 0.4920
Epoch 2/40
782/782 [==============================] - 94s 120ms/step - loss: 1.4202 - accuracy: 0.4860 - val_loss: 1.1817 - val_accuracy: 0.5753
Epoch 3/40
782/782 [==============================] - 94s 120ms/step - loss: 1.2574 - accuracy: 0.5479 - val_loss: 1.0560 - val_accuracy: 0.6257
Epoch 4/40
782/782 [==============================] - 94s 120ms/step - loss: 1.1465 - accuracy: 0.5897 - val_loss: 1.0090 - val_accuracy: 0.6400
Epoch 5/40
782/782 [==============================] - 101s 129ms/step - loss: 1.0788 - accuracy: 0.6171 - val_loss: 0.8934 - val_accuracy: 0.6844
Epoch 6/40
782/782 [==============================] - 96s 122ms/step - loss: 1.0210 - accuracy: 0.6411 - val_loss: 0.9206 - val_accuracy: 0.6845
Epoch 7/40
782/782 [==

In [4]:
# Here is our CIFAR-10 CNN ResNet.
inputs = tf.keras.Input(shape=(32, 32, 3), name='img')
x = Conv2D(32, 3, activation='relu')(inputs)
x = Conv2D(64, 3, activation='relu')(x)
block_1_output = MaxPooling2D(3)(x)

x = Conv2D(64, 3, activation='relu', padding='same')(block_1_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_2_output = tf.keras.layers.add([x, block_1_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_2_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_3_output = tf.keras.layers.add([x, block_2_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_3_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_4_output = tf.keras.layers.add([x, block_3_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_4_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_5_output = tf.keras.layers.add([x, block_4_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_5_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_6_output = tf.keras.layers.add([x, block_5_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_6_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_7_output = tf.keras.layers.add([x, block_6_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_7_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_8_output = tf.keras.layers.add([x, block_7_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_8_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_9_output = tf.keras.layers.add([x, block_8_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_9_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_10_output = tf.keras.layers.add([x, block_9_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_10_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_11_output = tf.keras.layers.add([x, block_10_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_11_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_12_output = tf.keras.layers.add([x, block_11_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_12_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_13_output = tf.keras.layers.add([x, block_12_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_13_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_14_output = tf.keras.layers.add([x, block_13_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_14_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_15_output = tf.keras.layers.add([x, block_14_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_15_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_16_output = tf.keras.layers.add([x, block_15_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_16_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_17_output = tf.keras.layers.add([x, block_16_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_17_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_18_output = tf.keras.layers.add([x, block_17_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_18_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_19_output = tf.keras.layers.add([x, block_18_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_19_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_20_output = tf.keras.layers.add([x, block_19_output])

x = Conv2D(64, 3, activation='relu')(block_20_output)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs, name='resnet')


model.compile(Adam(amsgrad=True), 'binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=256,
          epochs=10,
          validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 798s 20ms/sample - loss: 0.2837 - accuracy: 0.9003 - val_loss: 0.4043 - val_accuracy: 0.8999
Epoch 2/10
40000/40000 [==============================] - 703s 18ms/sample - loss: 0.2093 - accuracy: 0.9194 - val_loss: 0.4825 - val_accuracy: 0.8715
Epoch 3/10
40000/40000 [==============================] - 901s 23ms/sample - loss: 0.1708 - accuracy: 0.9341 - val_loss: 0.2555 - val_accuracy: 0.9088
Epoch 4/10
40000/40000 [==============================] - 957s 24ms/sample - loss: 0.1460 - accuracy: 0.9439 - val_loss: 0.1696 - val_accuracy: 0.9334
Epoch 5/10
40000/40000 [==============================] - 880s 22ms/sample - loss: 0.1289 - accuracy: 0.9509 - val_loss: 0.1831 - val_accuracy: 0.9315
Epoch 6/10
40000/40000 [==============================] - 717s 18ms/sample - loss: 0.1162 - accuracy: 0.9559 - val_loss: 0.1590 - val_accuracy: 0.9411
Epoch 7/10
40000/40000 [====================

In [9]:
# Create the CIFAR-100 CNN ResNet.
inputs = tf.keras.Input(shape=(32, 32, 3), name='img')
x = Conv2D(32, 3, activation='relu')(inputs)
x = Conv2D(64, 3, activation='relu')(x)
block_1_output = MaxPooling2D(3)(x)

x = Conv2D(64, 3, activation='relu', padding='same')(block_1_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_2_output = tf.keras.layers.add([x, block_1_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_2_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_3_output = tf.keras.layers.add([x, block_2_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_3_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_4_output = tf.keras.layers.add([x, block_3_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_4_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_5_output = tf.keras.layers.add([x, block_4_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_5_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_6_output = tf.keras.layers.add([x, block_5_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_6_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_7_output = tf.keras.layers.add([x, block_6_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_7_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_8_output = tf.keras.layers.add([x, block_7_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_8_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_9_output = tf.keras.layers.add([x, block_8_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_9_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_10_output = tf.keras.layers.add([x, block_9_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_10_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_11_output = tf.keras.layers.add([x, block_10_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_11_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_12_output = tf.keras.layers.add([x, block_11_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_12_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_13_output = tf.keras.layers.add([x, block_12_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_13_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_14_output = tf.keras.layers.add([x, block_13_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_14_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_15_output = tf.keras.layers.add([x, block_14_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_15_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_16_output = tf.keras.layers.add([x, block_15_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_16_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_17_output = tf.keras.layers.add([x, block_16_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_17_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_18_output = tf.keras.layers.add([x, block_17_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_18_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_19_output = tf.keras.layers.add([x, block_18_output])

x = Conv2D(64, 3, activation='relu', padding='same')(block_19_output)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
block_20_output = tf.keras.layers.add([x, block_19_output])

x = Conv2D(64, 3, activation='relu')(block_20_output)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(100, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs, name='resnet')


model.compile(Adam(amsgrad=True), 'binary_crossentropy', metrics=['accuracy'])

model.fit(x100_train, y100_train,
          batch_size=256,
          epochs=10,
          validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 776s 19ms/sample - loss: 0.0561 - accuracy: 0.9900 - val_loss: 0.0557 - val_accuracy: 0.9900
Epoch 2/10
40000/40000 [==============================] - 708s 18ms/sample - loss: 0.0545 - accuracy: 0.9900 - val_loss: 0.0542 - val_accuracy: 0.9900
Epoch 3/10
40000/40000 [==============================] - 674s 17ms/sample - loss: 0.0526 - accuracy: 0.9900 - val_loss: 0.0534 - val_accuracy: 0.9900
Epoch 4/10
40000/40000 [==============================] - 680s 17ms/sample - loss: 0.0505 - accuracy: 0.9900 - val_loss: 0.0511 - val_accuracy: 0.9900
Epoch 5/10
40000/40000 [==============================] - 682s 17ms/sample - loss: 0.0477 - accuracy: 0.9900 - val_loss: 0.0480 - val_accuracy: 0.9900
Epoch 6/10
40000/40000 [==============================] - 680s 17ms/sample - loss: 0.0448 - accuracy: 0.9900 - val_loss: 0.0469 - val_accuracy: 0.9900
Epoch 7/10
 2048/40000 [>...................

KeyboardInterrupt: 

In [17]:
# Create your own CNN that rivals a CNN ResNet for CIFAR-100.
batch_size = 20
num_classes = 100
epochs = 40
data_augmentation = True

model = Sequential()
model.add(Conv2D(120, (3, 3), padding='same', input_shape=x100_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(120, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(120, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(120, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x100_train, y100_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x100_test, y100_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        zca_epsilon=1e-06,
        rotation_range=0,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0., 
        horizontal_flip=True,
        vertical_flip=False, 
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

    datagen.fit(x100_train)

    model.fit(datagen.flow(x100_train, y100_train,
                        batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x100_test, y100_test))

scores = model.evaluate(x100_test, y100_test, verbose=0)



Using real-time data augmentation.
  ...
    to  
  ['...']
Train for 2500 steps, validate on 10000 samples
Epoch 1/40
2500/2500 [==============================] - 951s 381ms/step - loss: 4.1914 - accuracy: 0.0590 - val_loss: 3.7136 - val_accuracy: 0.1341
Epoch 2/40
2500/2500 [==============================] - 943s 377ms/step - loss: 3.6236 - accuracy: 0.1438 - val_loss: 3.2449 - val_accuracy: 0.2208
Epoch 3/40
2500/2500 [==============================] - 942s 377ms/step - loss: 3.3383 - accuracy: 0.1943 - val_loss: 2.9571 - val_accuracy: 0.2765
Epoch 4/40
2500/2500 [==============================] - 943s 377ms/step - loss: 3.1482 - accuracy: 0.2316 - val_loss: 2.8616 - val_accuracy: 0.2918
Epoch 5/40
2500/2500 [==============================] - 943s 377ms/step - loss: 3.0214 - accuracy: 0.2541 - val_loss: 2.7682 - val_accuracy: 0.3075
Epoch 6/40
2500/2500 [==============================] - 958s 383ms/step - loss: 2.9424 - accuracy: 0.2709 - val_loss: 2.6235 - val_accuracy: 0.3438
Epoc

KeyboardInterrupt: 